# Desenvolvimento de codigo
Apos o recebmento dos arquivos CONLL, foi feito a separação dos arquivos em suas determinadas revistas, em seguida foram separados os arquivos para as pastas separadas por areas de conhecimento. Apos esse processo foi iniciado o processo de codificação de uma atraves da plataforma JUPYTER para extrair as informações em arquivos ais facilmentes editaveis. Para isso forom utiizadas as segintes bibliotecas:

In [1]:
import numpy as np #Biblioteca de suporte de arrays e matriz multidimensionais
import pandas as pd #Biblioteca de manipulação e análise de dado
import os #Esse modulo prove alguns metodos para atuar com o sistema operacional, nesse projeto utilizado para gerenciamento de diretorios
import sys #Esse modulo prove acesso à algumas variaveis de sistema, nesse projeto utilizado para gerenciamento de diretorios
import matplotlib.pyplot as plt #Biblioteca padrão pala plotagem de graficos
import seaborn as sns #Biblioteca para melhoria de interface dos graficos
from pandas.io.json import json_normalize

# Desenvolvimento de funções.
    Em seguida foi desenvolvido o codigo necessario para a transformação dos arquivos CONLL em arquivos com dados analisaeis, seguindo as orientções de requisitos.

## getListCount
    Essa função recebe uma matriz de dados(df) e retorna uma lista com a contagem de inicidencia de uma coluna especifica(title).
    Utilizada para fazer a contagem dos frames

In [2]:
def getListCount(df,title):
    listOfterms = df[title].value_counts()
    return listOfterms

# returnHeaderAndContent
    Essa função recebe uma matriz de dados (df) retorna duas matrizes, uma com todos os dados referentes a primeira sentença  e outro com o restante do conteudo da matriz
    Essa função é executada duas vezes, uma vez para separar o titulo da revista e o resumo todo e em seguida o titulo do resumo e o conteudo

In [3]:
def returnHeaderAndContent(df):
    for index, row in df.iterrows():
        if(index != row['ID']-1):
            index_title = index
            break
    temp = index_title
    count = temp
    while temp == count:
        if(df.iloc[temp-1]['FORM'] != df.iloc[index_title-1]['FORM']):
            break
        count = count + index_title
        temp  = temp + index_title
    return  df.iloc[:temp-index_title].reset_index(drop=True), df.iloc[temp-index_title:].reset_index(drop=True)

## getPlot
Essa função recebe uma matriz de dados(df), um nome de uma coluna(label),e o nome do arquivo a ser salvo(filename).
Ele gera um grafico dos dados da matriz e salva esse grafico com o nome especificados.

In [4]:
def getPlot(df,label,fileName):
    sns.set(style="whitegrid")
    fig = plt.figure(figsize=(23, 10))
    fig.subplots_adjust(bottom=0.4)
    plt.xticks(rotation=70)
    try:
        ax = sns.barplot(x="index", y=label, data=df)
        fig.savefig(fileName)
    except:
        print("")
        
    plt.close(fig)

In [5]:
def  getData(dt):
    dfa = pd.DataFrame(columns=["ns","ne","token","target_name"])
    ns = 0
    for index, row in dt.iterrows():
        ne=1
        for token in row["tokens"]:
            dff = pd.DataFrame(columns=["ns","ne","token"],data=[[ns,ne,token]])
            dfa = dfa.append(dff).reset_index(drop=True)
            ne  = ne+1
        for frame in row["frames"]:
            ne    = json_normalize(json_normalize(frame)["target.spans"][0])["end"]
            text   = json_normalize(json_normalize(frame)["target.spans"][0])["text"]
            target = json_normalize(frame)["target.name"][0]
            dfa.loc[(dfa['ns'] == ns) & (dfa['ne'] == ne[0]) , 'target_name'] = target
        ns = ns+1
    return dfa

# 

In [11]:
from wordcloud import WordCloud
def getCloudWord(df,filen):
    dic ={}
    for index, row in df.iterrows():
        dic.update({row["index"]:row["target_name"]})
    wc = WordCloud().generate_from_frequencies(dic)
    sns.set(style="whitegrid")
    fig = plt.figure(figsize=(23, 10))
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.savefig(filen)

# Processo de extração dos dados
## Listagem e criação de pastas
    Utilizando a pasta do arquivo atual como base, inicialmente são listados todos os arquivos dessa pasta e adicionado na variavel directories.

In [7]:
path = "."
directories = os.listdir( path )

## Remoção de pastas não utilizadas
    São retirados os arquivos e diretorios que não são relevantes para a analise.

In [8]:
def cleanDirectory():
    try:
        for directory in directories:
            if(directory.startswith(".") or directory.endswith(".ipynb") or directory.endswith(".png") ):
                directories.remove(directory)
                cleanDirectory()
    except:
        print("")
cleanDirectory()

# Extração de dados

    É criado um laço de repetição passando por todos os diretorios do diretorio base e listado todos os arquivos desse diretorio que possuem a extensão CONLL.
    Dessa nova lista tambem é criado um novo laço de repetição para cada arquivo gera uma matriz que possue todos os dados do arquivo CONLL utilizando a função returnHeaderAndContent é possivel separar o titulo do journal do conteudo do resumo. Apos isso utilizando novamente a função returnHeaderAndContent é possivel separar o titulo do resumo do seu conteudo e separalos em matrizes diferentes.

In [16]:
columns = ["ns","ne","token","target_name"]
df_all_sum = pd.DataFrame(columns=columns)
df_head_sum = pd.DataFrame(columns=columns)
df_content_sum = pd.DataFrame(columns=columns)
df_title_sum  = pd.DataFrame(columns=columns)
df_tempo  = pd.DataFrame(columns=columns)
for directory in directories:
    try:
        makeDir = "..//exportsOfAbstracts//Disciplines//"+directory
        os.makedirs( makeDir );
    except:
        print("-")
    subDirectories = os.listdir( directory+"//" )
    for subDirectory in subDirectories:
        columns = ['ID', 'FORM' ,'LEMMA' ,'PLEMMA' ,'POS' ,'PPOS' ,
           'FEAT' ,'PFEAT' ,'HEAD' ,'PHEAD' ,'DEPREL' ,'PDEPREL','FILLPRED','PRED','APRED']
        listOfFiles = []
        folderPath = "{}".format(directory+"//"+subDirectory)
        
        files = os.listdir(folderPath)
        for fl in files:
            if(fl.endswith('.json')):
                listOfFiles.append("{}//{}".format(folderPath,fl))
        for i in listOfFiles:
            print(i)
            df_all = getData(pd.read_json(i, lines=True))
            df_title = df_all[df_all["ns"]==0]
            df_head =  df_all[df_all["ns"]==1]
            df_content = df_all[df_all["ns"]>1]
            df_title_sum =   df_title_sum.append(df_title)
            #df_tempo = df_head.append(df_content)
            df_all_sum =     df_all_sum.append(df_all) 
            df_head_sum =    df_head_sum.append(df_head) 
            df_content_sum = df_content_sum.append(df_content)
    filePath = "..//exportsOfAbstracts//Disciplines//"+directory+"//"+directory
    list_all = getListCount(df_all_sum,'target_name')[1:].to_frame().reset_index()
    list_head  = getListCount(df_head_sum,'target_name')[1:].to_frame().reset_index()
    list_content = getListCount(df_content_sum,'target_name')[1:].to_frame().reset_index()
    list_all.to_excel(filePath+"_list_all.xls", index = None, header=True)
    
    getPlot(list_all,"target_name",filePath+"_all.png")
    list_head.to_excel(filePath+"_list_head.xls", index = None, header=True)
    getPlot(list_head,"target_name",filePath+"_head.png")
    list_content.to_excel(filePath+"_list_content.xls", index = None, header=True)
    getPlot(list_content,"target_name",filePath+"_content.png")
    getCloudWord(list_all,filePath+"_cloud_word_all.png")
    getCloudWord(list_head,filePath+"cloud_word_head.png")
    getCloudWord(list_content,filePath+"cloud_word_content.png")

-
_COMPUTER//ieee-transactions-on-medical//2015-sep_JA_10-1109_TMI-2015-2409024_ieee-transactions-on-medical-imaging_zhao_yitian.seg.json
_COMPUTER//ieee-transactions-on-medical//2015-sep_JA_10-1109_TMI-2015-2409024_ieee-transactions-on-medical-imaging_zhao_yitian.seg.json
_COMPUTER//ieee-transactions-on-medical//2016-may_JA_10-1109_TMI-2015-2482920_ieee-transactions-on-medical-imaging_roth_holger.seg.json
_COMPUTER//ieee-transactions-on-medical//2015-sep_JA_10-1109_TMI-2015-2409024_ieee-transactions-on-medical-imaging_zhao_yitian.seg.json
_COMPUTER//ieee-transactions-on-medical//2016-may_JA_10-1109_TMI-2015-2482920_ieee-transactions-on-medical-imaging_roth_holger.seg.json
_COMPUTER//ieee-transactions-on-medical//2016-may_JA_10-1109_TMI-2016-2526689_ieee-transactions-on-medical-imaging_van-grinsven_mark.seg.json
_COMPUTER//ieee-transactions-on-medical//2015-sep_JA_10-1109_TMI-2015-2409024_ieee-transactions-on-medical-imaging_zhao_yitian.seg.json
_COMPUTER//ieee-transactions-on-medical/

KeyboardInterrupt: 

Utilizando essas matrizes recem criadas é utilizada a função getListCount para obter a contagem dos frames.

In [ ]:
list_all = getListCount(df_all_sum,'PRED')[1:].to_frame().reset_index()
list_head  = getListCount(df_head_sum,'PRED')[1:].to_frame().reset_index()
list_content = getListCount(df_content_sum,'PRED')[1:].to_frame().reset_index()

Apos a criação da lista de abstracts foi utilizada a função da biblioteca do pandas to_excel para criar o documento de cada uma das listas. E finalizando tambem foi criado os graficos de cada uma das listas.

In [42]:
list_all.to_excel("_list_all.xls", index = None, header=True)
getPlot(list_all,"PRED","_all.png")
list_head.to_excel("_list_head.xls", index = None, header=True)
getPlot(list_head,"PRED","_head.png")
list_content.to_excel("_list_content.xls", index = None, header=True)
getPlot(list_content,"PRED","_content.png")
getCloudWord(list_all,"wc_content_all.png")
getCloudWord(list_head,"wc_content_head.png")
getCloudWord(list_content,"wc_content_content.png")

In [ ]:
['2015-sep_JA_10-1109_TMI-2015-2409024_ieee-transactions-on-medical-imaging_zhao_yitian.seg.json', 
 '2016-may_JA_10-1109_TMI-2015-2482920_ieee-transactions-on-medical-imaging_roth_holger.seg.json', 
 '2016-may_JA_10-1109_TMI-2016-2526689_ieee-transactions-on-medical-imaging_van-grinsven_mark.seg.json',
 '2016-jan_JA_10-1109_TMI-2015-2458702_ieee-transactions-on-medical-imaging_xu_jun.seg.json',
 'desktop.ini',
 '2016-may_JA_10-1109_TMI-2016-2535302_ieee-transactions-on-medical-imaging_tajbakhsh_nima.seg.json',
 '2016-may_JA_10-1109_TMI-2016-2528129_ieee-transactions-on-medical-imaging_dou.seg.json', 
 '2016-may_JA_10-1109_TMI-2016-2548501_ieee-transactions-on-medical-imaging_moeskops_pim.seg.json', 
 '2016-may_JA_10-1109_TMI-2016-2536809_ieee-transactions-on-medical-imaging_setio_arnaud.seg.json', 
 '2015-may_JA_10-1109_TMI-2014-2375065_ieee-transactions-on-medical-imaging_ferguson_matthew.seg.json', 
 '2016-may_JA_10-1109_TMI-2016-2538465_ieee-transactions-on-medical-imaging_pereira_sergio.seg.json', 
 '2016-nov_JA_10-1109_TMI-2016-2546227_ieee-transactions-on-medical-imaging_liskowski_pawel.seg.json', 
 '2016-may_JA_10-1109_TMI-2016-2525803_ieee-transactions-on-medical-imaging_sirinukunwattana_korsuk.seg.json',
 '2015-feb_JA_10-1109_TMI-2014-2359650_ieee-transactions-on-medical-imaging_christensen-jeffries_kirsten.seg.json',
 '2016-jan_JA_10-1109_TMI-2015-2457891_ieee-transactions-on-medical-imaging_li_qiaoliang.seg.json',
 '2015-nov_JA_10-1109_TMI-2015-2428634_ieee-transactions-on-medical-imaging_demene_charlie.seg.json', 
 '2015-oct_JA_10-1109_TMI-2014-2377694_ieee-transactions-on-medical-imaging_menze_bjoern.seg.json', 
 '2016-may_JA_10-1109_TMI-2016-2528162_ieee-transactions-on-medical-imaging_shin_hoo-chang.seg.json',
 '2015-feb_JA_10-1109_TMI-2014-2361481_ieee-transactions-on-medical-imaging_zhang_xiaofan.seg.json', 
 '2016-may_JA_10-1109_TMI-2016-2535865_ieee-transactions-on-medical-imaging_anthimopoulos_marios.seg.json', 
 '2016-may_JA_10-1109_TMI-2016-2532122_ieee-transactions-on-medical-imaging_kallenberg_michiel.seg.json']